# Monte Carlo Integration and Implementation on VegasFlow
## Introduction:
### In this project, we will have some brief tutorial to Monte Carlo Integration which is a broad class of computational algorithms that rely on repeated random sampling to obtain numerical results. 
### VegasFlow is developed with a focus on speed and efficiency, enabling researchers to perform very expensive calculation as quick and easy as possible. We will implement Vegasflow with MC integration setup at the end. With the computation being timed, we will able to see how efficient this package is. 



In [4]:
# Import cell
%matplotlib inline

import numpy as np
from scipy.special import gamma
import matplotlib.pyplot as plt

## Spheres in higher dimensions

Monte Carlo integration really shines for integrals with a high number of dimensions.  Let's investigate with a simple calculation: finding the volume of a unit sphere (radius $R=1$) in $d$ dimensions.  This will be a good test case because we know the answer exactly:

$$
V_d(1) = \frac{\pi^{d/2}}{\Gamma(d/2+1)}
$$

where $\Gamma$ is the Euler gamma function (available as the `gamma` function from `scipy.special` - I've imported it above.)  For even $d=2k$, the formula simplifies to

$$
V_{2k}(1) = \frac{\pi^k}{k!}
$$

The $d=2$ sphere is just the circle, and we recognize $V_2 = \pi$.  The next (probably unfamiliar) example is $d=4$, for which we get $V_4 = \pi^2 / 2$.


### Part A

Now let's set the problem up as an integral.  In $d=2$, the equation for a circle is $x^2 + y^2 = 1$.  If we restrict to the first quadrant $x>0, y>0$, we can rewrite this as $y = \sqrt{1-x^2}$ with no ambiguity, and then we have

$$
V_2(1) = 4 \int_0^1 dx\ \sqrt{1-x^2} \approx 4V \langle \sqrt{1 - x^2} \rangle = 4 \langle \sqrt{1-x^2} \rangle = \frac{4}{N} \sum_{i=1}^N \sqrt{1-x_i^2}
$$

where we draw $N$ random points $x_i$, and the 4 is to compensate for restricting to the first quadrant.  Here $V$ is the volume of the sampling region, which is just 1 (the length of the $x$-interval.)



In [5]:
N=1000  # Number of samples

# Draw Ns random numbers in [0,1] with np.random.rand()

# Find E1, the mean of sqrt(1-x^2) over the random numbers x

# Integral V2 is| equal to 4 times E1

x = np.random.rand(N)
E1 = np.mean(np.sqrt(1-x**2))
V2 = 4*E1

print(V2) 



3.142451281987086


This should give us something reasonably close to the expected answer of $\pi$, but probably slightly off due to random fluctuations. 



### Part B

Now we're ready to deal with the general case.  For any number of dimensions $d$, we can write the volume of a unit sphere as the integral


![alt text](formula1.png)


where the $2^d$ comes from picking the higher-dimensional equivalent of the first quadrant.  The limits of integration get a bit tricky in higher dimensions, unfortunately!  Although it's not too bad to deal with, let's start with the conceptually simpler algorithm instead.  Instead of eliminating $x_d$ and getting the square root, we can simply write

$$
V_d(1) = \int dx_1 ... dx_d\ \Theta(1 - \sum_{i=1}^{d} x_i^2)
$$
where $\Theta$ is the Heaviside step function.  This leads us back to the "hit-or-miss" Monte Carlo algorithm, since the step function is just 1 for any point inside the unit sphere and 0 outside.

We would compute the volume of the `d`-sphere using `N` samples and the hit-or-miss algorithm:

1. Draw points $(x_1, x_2, ..., x_d)$ randomly over the hypercube ($-1 \leq x_i \leq 1$).
2. Count the number of points $N_{\rm hit}$ inside the unit sphere, $\sum_{i=1}^d x_i^2 \leq 1$.
3. Return $V_d = V (N_{\rm hit} / N)$, where $V = 2^d$ is the volume of the hypercube we're sampling from.


In [6]:
def V_sphere_MC(d, N):
    
    # Draw an (N x d) array containing N random vectors of length d
    # The random numbers should be between [-1,1], so rescale them!
    
    # Square the random array, then sum over the axis of length d, 
    # to produce an array of squared vector lengths
    
    # Use the squared lengths to create a boolean array which is True
    # for any points inside the sphere
    
    # Use np.count_nonzero on the boolean array,
    # to count N_hit, how many points are in the sphere
    
    # Vd = (2**d) * N_hit / N
    
    x = 2 * np.random.rand(N, d) - 1
    
    in_sphere = np.sum(x**2, axis=1) <= 1
    N_hit = np.count_nonzero(in_sphere)
    
    Vd = 2**d * N_hit / N
   
    
    
    return Vd
    

V_sphere_MC(4, 1000)

4.704

In [7]:
# Evaluate in d=2: should be close to pi = 3.1415...
print(V_sphere_MC(2, 10000))

# Evaluate in d=3: should be close to (4/3) pi = 4.18879...
print(V_sphere_MC(3, 10000))

3.142
4.244


The standard error formula still works for hit-or-miss Monte Carlo!  Since the Heaviside function is equal to its own square, we just have

$$
\sigma_{d, \rm sem} = \sqrt{\frac{2^d I - I^2}{N}}
$$


If we try this with $d=2$, how does hit-or-miss compare to the standard error we got in part B?

In [8]:
def V_sphere_witherr(d, N):
    Vd = V_sphere_MC(d,N)
    sigma_SEM = np.sqrt((2**d * Vd - Vd*Vd) / N)
    
    return (Vd, sigma_SEM)
    

In [9]:
# Testing in d=2 again; should (on most trials) be within 1 sigma_SEM of pi.
# Error is a bit larger for the same number of points, since we're sampling one extra dimension!
V_sphere_witherr(2,1000)

(3.08, 0.05323156958046607)

### Part C

Now that we have a general code working, let's start to turn up the number of dimensions and see how the _curse of dimensionality_ sets in. 


In [10]:
d = 10

In [11]:
def exact_vol(d):
    return np.pi**(d/2) / gamma(d/2 + 1)

print("Exact volume: V_{0:d} = {1:g}".format(d, exact_vol(d)))
print("100 samples: V_{0:d} = {1:g} +/- {2:g}".format(d, *V_sphere_witherr(d, 100)))
print("1000 samples: V_{0:d} = {1:g} +/- {2:g}".format(d, *V_sphere_witherr(d, 1000)))
print("10000 samples: V_{0:d} = {1:g} +/- {2:g}".format(d, *V_sphere_witherr(d, 10000)))
print("100000 samples: V_{0:d} = {1:g} +/- {2:g}".format(d, *V_sphere_witherr(d, 100000)))
print("1000000 samples: V_{0:d} = {1:g} +/- {2:g}".format(d, *V_sphere_witherr(d, 1000000)))

Exact volume: V_10 = 2.55016
100 samples: V_10 = 0 +/- 0
1000 samples: V_10 = 1.024 +/- 1.02349
10000 samples: V_10 = 3.2768 +/- 0.578334
100000 samples: V_10 = 2.51904 +/- 0.16041
1000000 samples: V_10 = 2.46682 +/- 0.0501989


__The qualitative change we should have noticed around `d=10` or so is due to a simple quirk of higher-dimensional geometry.  We're sampling from a hypercube enclosing the unit sphere, which has volume__

$$
V_{d,\rm{cube}} = 2^d
$$

__We know from 2 and 3 dimensions that the unit sphere fits _inside_ this cube, of course (that's why we can use the cube as a sampling region) - only the corners of the cube are outside the sphere.  But how much space is that?  The ratio of sphere to cube volume is, in even $d$ for simplicity,__

$$
\frac{V_{2k,\rm{sphere}}}{V_{2k,\rm{cube}}} = \frac{\pi^k}{4^k k!}
$$

__This is dropping to zero really, really fast - for $k=5$ ($d=10$) the sphere only takes up 0.2% of the cube, and for $d=20$ the cube's volume is about $10^8$ times larger!  (The difference is still just due to the corners of the cube, but a hypercube has a _lot_ of corners.)__

### Part D

Monte Carlo integration is most often used for high-dimensional integrals.  But another convenient application can be to integrals that are just really messy to set up due to their geometry.  For example, suppose we want to calculate the mass of a steel sphere that has a cylindrical hole of radius $r$ bored through the center:

![MC](MC.png)

Assuming a constant density, we just need to know the volume to find the mass.  We can write the equation defining the sphere as:

$$
\begin{cases}
\sqrt{x^2 + y^2 + z^2} &\leq 1, \\
\sqrt{x^2 + y^2} &\geq 0.3.
\end{cases}
$$

This would be annoying to set up by hand, since we have a mixture of spherical and cylindrical symmetry.  But finding the volume using hit-or-miss Monte Carlo is easy - we just generate random points in the cube $-1 \leq x,y,z \leq 1$, and then see if they're in the sphere (and not in the cylindrical hole).

We modify our hit-and-miss volume Monte Carlo above to find the volume of this shape in three dimensions.

In [12]:
def V_sphere_with_hole_MC(N):
    ### BEGIN SOLUTION
    # Draw random numbers in the cube -1 <= x,y,z <= 1
    x = 2 * np.random.rand(N, 3) - 1
    
    
    sphere_mask = np.sum(x**2, axis=1) <= 1
    cyl_mask = np.sum(x[:,0:2]**2, axis=1) >= 0.3**2
        
    # Count how many points are True for _both_ masks
    N_inside = np.count_nonzero(np.logical_and(sphere_mask, cyl_mask))
    
    return 2**3 * N_inside / N
   


Now run the Monte Carlo and find the volume.  In this case, we have a theoretical estimate for the result:

$$
V = V_{\rm sphere} - V_{\rm cyl} \\
= \frac{4}{3} \pi R^3 - \pi r^2 h \\
= \frac{4}{3} \pi - \pi (0.3)^2 (2) \\
\approx 3.62.
$$

This isn't _quite_ right, because there are a handful of points that are in the cylinder but not in the sphere which are being oversubtracted; we might try using our Monte Carlo simulation to see how much volume that region (in the cylinder, out of the sphere) really is.  But it should be pretty close.

For volume this exercise is sort of trivial, but if the object wasn't constant density, or if we wanted some more complicated property like the moments of inertia, then Monte Carlo would be a nice way to do the integral!  

In [62]:
V_sphere_with_hole_MC(100000)

3.62704

### Part E

Monte Carlo integration is used as a fast way to numerically estimate integrals which cannot be done analytically, and the VEGAS algorithm approximates the exact distribution by making a number of passes over the integration region while histogramming the function f. Vegas locates the portions of the integrand that contribute most to the integral, and then it preferentially samples those areas.

__We will first integrate MC with Vegasflow by using the CFFI library:__

In [13]:
import time
import numpy as np
from vegasflow.configflow import DTYPE
from vegasflow.vflow import VegasFlow
import tensorflow as tf

from cffi import FFI
ffibuilder = FFI()


# MC integration setup
dim = 4
ncalls = np.int32(1e5)
n_iter = 5

if DTYPE is tf.float64:
    C_type = "double"
elif DTYPE is tf.float32:
    C_type = "float"
else:
    raise TypeError(f"Datatype {DTYPE} not understood")


ffibuilder.cdef(f"""
    void symgauss({C_type}*, int, int, {C_type}*);
""")

ffibuilder.set_source("_symgauss_cffi", f"""
    void symgauss({C_type} *x, int n, int evts, {C_type}* out)
    {{
        for (int e = 0; e < evts; e++)
        {{
            {C_type} a = 0.1;
            {C_type} pref = pow(1.0/a/sqrt(M_PI), n);
            {C_type} coef = 0.0;
            for (int i = 1; i <= 100*n; i++) {{
                coef += ({C_type}) i;
            }}
            for (int i = 0; i < n; i++) {{
                coef += pow((x[i+e*n] - 1.0/2.0)/a, 2);
            }}
            coef -= 100.0*n*(100.0*n+1.0)/2.0;
            out[e] = pref*exp(-coef);
        }}
    }}
""")
ffibuilder.compile(verbose=True)

from _symgauss_cffi import ffi, lib

def symgauss(xarr, n_dim=None, **kwargs):
    if n_dim is None:
        n_dim = xarr.shape[-1]
    n_events = xarr.shape[0]

    res = np.empty(n_events, dtype = DTYPE.as_numpy_dtype)
    x_flat = xarr.numpy().flatten()

    pinput = ffi.cast(f'{C_type}*', ffi.from_buffer(x_flat))
    pres = ffi.cast(f'{C_type}*', ffi.from_buffer(res))
    lib.symgauss(pinput, n_dim, n_events, pres)
    return res

if __name__ == "__main__":
    """Testing a basic integration"""

    print(f"VEGAS MC, ncalls={ncalls}:")
    start = time.time()
    vegas_instance = VegasFlow(dim, ncalls)
    vegas_instance.compile(symgauss, compilable = False)
    r = vegas_instance.run_integration(n_iter)
    end = time.time()
    print(f"time (s): {end-start}")


VEGAS MC, ncalls=100000:


NameError: name 'VegasFlow' is not defined

Output: ![Output](cffi.png)


__We can then see the other example using the vegas wrapper helper.__

In [0]:
from vegasflow.configflow import DTYPE
import time
import numpy as np
import tensorflow as tf
from vegasflow.vflow import vegas_wrapper
from vegasflow.plain import plain_wrapper 


# MC integration setup
dim = 4
ncalls = np.int32(1e5)
n_iter = 5


@tf.function
def symgauss(xarr, n_dim=None, **kwargs):
    """symgauss test function"""
    if n_dim is None:
        n_dim = xarr.shape[-1]
    a = tf.constant(0.1, dtype=DTYPE)
    n100 = tf.cast(100 * n_dim, dtype=DTYPE)
    pref = tf.pow(1.0 / a / np.sqrt(np.pi), n_dim)
    coef = tf.reduce_sum(tf.range(n100 + 1))
    coef += tf.reduce_sum(tf.square((xarr - 1.0 / 2.0) / a), axis=1)
    coef -= (n100 + 1) * n100 / 2.0
    return pref * tf.exp(-coef)


if __name__ == "__main__":
    """Testing several different integrations"""
    print(f"VEGAS MC, ncalls={ncalls}:")
    start = time.time()
    ncalls = 10*ncalls
    r = vegas_wrapper(symgauss, dim, n_iter, ncalls)
    end = time.time()
    print(f"Vegas took: time (s): {end-start}")



Output: ![alt text](wrapper.png)

# Question:
What is the accuracy of the Vegas algorithm when finding the peaks of the integrand?

# Group Project Idea:
I think solving the time-independent Schrodinger equation for ground state solution would be fun or solving V(x) for an infinite square well. 

## Citations
   

@misc{juacrumar_carrazza_stefano_cruz-martinez_2020, title={N3PDF/vegasflow: VegasFlow 1.2.0}, url={https://doi.org/10.5281/zenodo.3691926}, journal={Zenodo}, author={Juacrumar and Carrazza and Stefano and Cruz-Martinez}, year={2020}, month={Sep}} 

 @misc{carrazza_cruz-martinez_2020, title={VegasFlow: accelerating Monte Carlo simulation across multiple hardware platforms}, url={https://arxiv.org/abs/2002.12921}, journal={arXiv.org}, author={Carrazza, Stefano and Cruz-Martinez, Juan M.}, year={2020}, month={May}} 